<a href="https://colab.research.google.com/github/gihoonpark/Novelist_classification-NLP/blob/main/%EC%86%8C%EC%84%A4%EC%9E%91%EA%B0%80_%EB%B6%84%EB%A5%98ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
!pip install tensorflow-gpu==1.14  # GPU

In [ ]:
!pip install nltk

Library

In [ ]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional, Input, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
# 코사인 유사도, 로지스틱
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
import nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

Load Data

In [ ]:
trn_file = '/content/drive/MyDrive/dataset/소설작가 ai/train.csv'
tst_file = '/content/drive/MyDrive/dataset/소설작가 ai/test_x.csv'
sample_file = '/content/drive/MyDrive/dataset/소설작가 ai/sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [ ]:
train = pd.read_csv(trn_file, index_col=0)
train.head()

In [ ]:
test = pd.read_csv(tst_file, index_col=0)
test.head()

In [ ]:
# 코사인 유사도, 로지스틱
trn = train
tst = test

trn['text'] = trn['text'].str.lower().apply(alpha_num)
tst['text'] = tst['text'].str.lower().apply(alpha_num)

s = str(trn.text[:])
tokens = word_tokenize(s)
print(tokens)
lemmatizer = WordNetLemmatizer()
# stemmer = SnowballStemmer("english")

vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

In [ ]:
# 코사인 유사도, 로지스틱
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
y = trn.author.values
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    print(f'training model for CV #{i_cv}')
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

In [ ]:
# 코사인 유사도, 로지스틱
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

In [ ]:
def remove_stopwords_nltk(text): # 2.불용어 제거
    final_text = []
    for i in text.split(): # text를 띄어쓰기 기준으로 원소로 나누고, 하나 씩 불용어와 비교, 불용어가 아니면 final_text에 하나씩 추가.
        if i.strip().lower() not in stop_words: # 원소를 소문자로 바꾼 후, 불용어와 비교.
            final_text.append(i.strip())
    return " ".join(final_text) # 띄어쓰기 기준으로 나눈 것을 다시 문장으로 연결하여 출력.

preprocessing

In [ ]:
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text) # 1.정제 : [^A-Za-z0-9 ]빼고 전부 없애기. -> r'[^A-Za-z0-9 ]'

In [ ]:
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords_nltk)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords_nltk)

In [ ]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

In [ ]:
X_train[:3]

In [ ]:
vocab_size = 20000
embedding_dim = 64
max_length = 505
padding_type='post'

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)# 토큰화 후
word_index = tokenizer.word_index # 정수 인코딩

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length) # 모든 입력 데이터 크기 통일시키기(padding 방법, max 길이 설정)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length) 
print(trn.shape, tst.shape)

Model & Training

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query):
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
def get_model():
    sequence_input = Input(shape=(max_length,), dtype='int32')
    embedded_sequences = Embedding(vocab_size, embedding_dim, input_length=max_length, mask_zero = True)(sequence_input)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(embedded_sequences)
    
    state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = Concatenate()([forward_c, backward_c]) # 셀 상태
    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)
    
    output = Dense(n_class, activation="softmax")(context_vector)
    model = Model(inputs=sequence_input, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1): # cv

    print(f'training model for CV #{i}')
    #clf = get_model()
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
    sequence_input = Input(shape=(max_length,), dtype='int32')
    embedded_sequences = Embedding(vocab_size, embedding_dim, input_length=max_length, mask_zero = True)(sequence_input)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(embedded_sequences)
    
    state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = Concatenate()([forward_c, backward_c]) # 셀 상태
    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)

    dense = Dense(20, activation="relu")(context_vector)
    output = Dense(n_class, activation="softmax")(dense)
    model = Model(inputs=sequence_input, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    model.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=5,
            batch_size=512,
            callbacks=[es])

    p_val[i_val, :] = model.predict(trn[i_val])
    p_tst += model.predict(tst) / n_fold

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100 :8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Submit

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/dataset/소설작가 ai/submission_소설작가1.csv', index_col=0)
df2 = pd.read_csv('/content/drive/MyDrive/dataset/소설작가 ai/submission_소설작가2.csv', index_col=0)
df3 = pd.read_csv('/content/drive/MyDrive/dataset/소설작가 ai/submission_소설작가3.csv', index_col=0)

In [ ]:
df4 = (df1 + df2 + df3) / 3
df4

In [ ]:
df4.to_csv(sample_file)

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

In [ ]:
sub[sub.columns] = p_tst
sub.head()

In [ ]:
sub.to_csv(sample_file)